In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
drive.mount('/content/drive')
!unzip -n '/content/drive/MyDrive/DL_flowers.zip' >> /dev/null

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = 'flowers/'
img_path = data_path + 'images/'
label_path = data_path + 'labels.csv' 

In [ ]:
labels = pd.read_csv(label_path, dtype ={"class": "category"} )

In [ ]:
labels

,image_name,class
0,100080576_f52e8ee070_n.jpg,0
1,10140303196_b88d3d6cec.jpg,0
2,10172379554_b296050f82_n.jpg,0
3,10172567486_2748826a8b.jpg,0
4,10172636503_21bededa75_n.jpg,0
...,...,...
4310,9831362123_5aac525a99_n.jpg,4
4311,9870557734_88eb3b9e3b_n.jpg,4
4312,9947374414_fdf1d0861c_n.jpg,4
4313,9947385346_3a8cacea02_n.jpg,4


In [ ]:
train_data, test_data = train_test_split(labels, test_size=0.3, random_state = 0)
dev_data, test_data = train_test_split(test_data, test_size = 50, random_state = 0) 

In [ ]:
train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)
dev_data = dev_data.reset_index(drop = True)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
classes = ['0','1','2','3','4']
n_clases = len(classes)
img_width = img_height = 224
x_col = 'image_name'
y_col = 'class'

In [ ]:
batch_size = 128
train_generator = datagen.flow_from_dataframe(
    dataframe = train_data, 
    directory = img_path, 
    x_col=x_col, y_col=y_col, 
    class_model = 'categorical', 
    target_size = (img_width, img_height), 
    batch_size=batch_size, 
    classes = classes)
test_generator = datagen.flow_from_dataframe(dataframe = test_data, directory = img_path, x_col=x_col, y_col=y_col, class_model = 'categorical', target_size = (img_width, img_height), batch_size=batch_size, classes = classes)
dev_generator = datagen.flow_from_dataframe(dataframe = dev_data, directory = img_path, x_col=x_col, y_col=y_col, class_model = 'categorical', target_size = (img_width, img_height), batch_size=batch_size, classes = classes)

Found 3020 validated image filenames belonging to 5 classes.
Found 50 validated image filenames belonging to 5 classes.
Found 1245 validated image filenames belonging to 5 classes.


In [ ]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
predictions = tf.keras.layers.Dense(n_clases, activation='softmax')(x)


In [ ]:
model = tf.keras.models.Model(inputs=[base_model.input], outputs=[predictions])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    epochs= 10,
    verbose=2,
    steps_per_epoch=len(train_data)/batch_size,
    validation_data=dev_generator,
    validation_steps = len(dev_data)/batch_size
)

Epoch 1/10
23/23 - 26s - loss: 1.5362 - accuracy: 0.3152 - val_loss: 1.3191 - val_accuracy: 0.5036 - 26s/epoch - 1s/step
Epoch 2/10
23/23 - 19s - loss: 1.1612 - accuracy: 0.6079 - val_loss: 1.0640 - val_accuracy: 0.6386 - 19s/epoch - 808ms/step
Epoch 3/10
23/23 - 19s - loss: 0.9525 - accuracy: 0.7046 - val_loss: 0.9177 - val_accuracy: 0.6940 - 19s/epoch - 812ms/step
Epoch 4/10
23/23 - 19s - loss: 0.8267 - accuracy: 0.7421 - val_loss: 0.8259 - val_accuracy: 0.7261 - 19s/epoch - 815ms/step
Epoch 5/10
23/23 - 19s - loss: 0.7422 - accuracy: 0.7679 - val_loss: 0.7589 - val_accuracy: 0.7430 - 19s/epoch - 815ms/step
Epoch 6/10
23/23 - 19s - loss: 0.6817 - accuracy: 0.7887 - val_loss: 0.7101 - val_accuracy: 0.7614 - 19s/epoch - 806ms/step
Epoch 7/10
23/23 - 19s - loss: 0.6341 - accuracy: 0.8003 - val_loss: 0.6727 - val_accuracy: 0.7791 - 19s/epoch - 812ms/step
Epoch 8/10
23/23 - 19s - loss: 0.5951 - accuracy: 0.8159 - val_loss: 0.6414 - val_accuracy: 0.7855 - 19s/epoch - 815ms/step
Epoch 9/10


In [ ]:
len(test_data)/batch_size

0.390625

In [ ]:
test_loss, test_acc = model.evaluate(test_generator,steps=0.390625, verbose = 1)
print(f"test loss: {test_loss}, test_acc: {test_acc}" )

TypeError: ignored